<a href="https://colab.research.google.com/github/abtheo/SolarPanelEstimation/blob/master/SolarPanelPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predicting Solar Panel Production 🌞


---


#Mission Statement 🎯

---


> ***Wouldn't it be great if could predict the power output of solar panels, without paying to install an on-site meter?***


Every solar panel owner would love to know how much power their panels are producing for them. Unfortunately, this knowledge currently comes at the cost of installing a physical electricity meter on-site to read from the panels directly. Fortunately though, every site *already has* an electricity meter installed, which measures the total energy delivered and returned to the national grid. 


That's where we come in! We use only data from the existing grid meters, combined with freely available meteorological data (courtesy of the [Koninklijk Nederlands Meteorologisch Instituut](https://https://www.knmi.nl)). 


#Setup ⚙


---

>Run this section upon launching the notebook to load all the required dependencies.



*Though all code blocks contain explanatory comments, it would be helpful to have a basic understanding of the following libraries:*

*   [*Numpy*](https://numpy.org)
*   [*Pandas*](https://pandas.pydata.org/)



##Set File Directory

In [ ]:
"""Setting Root Directory:
Change root_dir to your relative path to the tutorial folder."""
!git clone "{}" ./temp      # clone github repository to temp folder
!mv ./temp/* "{PROJECT_PATH}"       # move all files/folders in temp folder to folder defined in project path
!rm -rf ./temp                      # remove all the files/folders in temp folder
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./   # use remote sync to copy from google drive to local runtime google colab
                                                    # but exclude data folder
                                                    # https://www.computerhope.com/unix/rsync.htm



##Import Dependencies
Python libraries utilised throughout the notebook.

In [ ]:
#Standard Python libraries
import csv
import math
import random
import datetime

#Data Science / plotting libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
%pip install ann_visualizer #Drawing package for neural networks, see: https://github.com/Prodicode/ann-visualizer
from ann_visualizer.visualize import ann_viz
import graphviz

#File interactions
import pickle
import glob

#High-level Machine Learning API for TensorFlow.
#See: https://keras.io
# #%tensorflow_version 2.x
#Suppress TF warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras import regularizers
from keras.callbacks import *

#Sci-kit Learn, implements manly helpful tools for ML
#See: https://scikit-learn.org/
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import tree

#Supervised Learning - Overview 👨‍🏫

---

At heart, we face a **supervised learning problem**. This field is a subset of machine learning and AI as a whole, and tackles any problem which can be framed in the following way:

>***Given enough examples of some input data, and desired output data; If a relationship exists between input and output, a predictive model can be trained to approximate this relationship.***

The "input data" we have initially is readings from electricity meters for a given location. Specifically, the total electricity Delivery and Return (in KWh) read through the meter, on an hourly basis. These two input measurements are known collectively as **features**, and are denoted mathemetically as  $x$ .

The desired "output data" is the electricity production of solar PV panels located at the target site (again in KWh). Known as the **target** or **label**, it is denoted mathematically as $y$ .


We work under the assumption that after suitable *feature engineering*, there exists some suitable function $f$ which satisfies the equation:

>$f(x) = y$


Therefore, learning an approximation of $f$ will allow us to predict the desired targets $y$ when provided with unseen examples of input data $x$. 

A **predictive model** does precisely this; it is a specialised implementation of $f(x)$, fine-tuned to your specific problem domain to give realistic predictions of the targets $y$.

##Importing Raw Data
Let's take a look at an example dataset of our features and targets. This data comes from Havelte, at a solar production site.



*   Features ($x$) are Delivery and Return.
*   Target ($y$) is Back Production. (BPM)
*   Date and Timestamp are when the measurement was made.



In [ ]:
#Section 1 uses the raw readings subdirectory, so set to the current data_dir.
data_dir = root_dir + "Raw_Readings/"

#Read data from an example CSV file as a Pandas DataFrame.
dataset = pd.read_csv(data_dir+"EnergyAndPV_0.csv", sep=';', decimal=',') 

#Take a look at a few rows of data
dataset.iloc[10:20]

##Data Visualisation
It's helpful to start by plotting our data to get a better idea of what we're working with. In particular, we're looking for relationships between the features and the target.

In [ ]:
"""Plot features and targets against timestep iterations on the same axes."""
X = list(range(len(dataset)))

fig = go.Figure()

#Plot BackProduction
fig.add_trace(go.Scatter(
                x=X,
                y=dataset['BackProduction'],
                name='Solar PV Production',
                line_color='mediumseagreen',
                opacity=0.8))

#Plot Return
fig.add_trace(go.Scatter(
                x=X,
                y=dataset['Return'],
                name='Electricity Returned',
                line_color='red',
                opacity=0.8))

#Plot Delivery
fig.add_trace(go.Scatter(
                x=X,
                y=dataset['Delivery'],
                name='Delivered Electricity',
                line_color='royalblue',
                opacity=0.8))

#Make it fancy!
fig.update_layout(go.Layout(
    title=dict(x=0.45),
    yaxis=dict(title='Energy (KWh)'),
    xaxis=dict(title='Timestep'),
    title_text="Havelte - Back Production vs Return",
    hovermode="x",
    template="plotly_dark"))
fig.show()

##Correlations & Heatmap
Checking correlations and their coefficients is also a useful metric.
If a feature has a high correlation with the target, it *should* be a useful feature.

However, features should not *necessarily* be discarded
just because of a low correlation in 1 dimension,
as they could potentially be useful when combined in higher dimensional space.


*Recommended Reading: [Correlation Coefficients](https://blog.bigml.com/2015/09/21/looking-for-connections-in-your-data-correlation-coefficients/)*

In [ ]:
"""Pearson Correlation Heatmap
— Closer to 0 implies weaker correlation (exact 0 implying no correlation)
— Closer to 1 implies stronger positive correlation
— Closer to -1 implies stronger negative correlation"""
cmap = plt.cm.coolwarm
sns.heatmap(dataset.corr(), annot=True, cmap=cmap,  fmt='g')

#Print correlation coefficients numerically
dataset.corr()

Furthermore, we can create a [pair-plot](https://seaborn.pydata.org/generated/seaborn.pairplot.html), whereby each column of data is plotted against one another. Once again, we're especially interested in linear correlations between the targets and features.


In [ ]:
"""Plot features against one another in a pair grid. """
#Using Seaborn for visualisation. For details, see: https://seaborn.pydata.org/
g = sns.PairGrid(dataset)
g.map(plt.scatter)

##Comparing Multiple Datasets
Havelte looks promising! We can see that the feature 'Return' has an extremely strong correlation to the target, 'BackProduction'. This is good evidence that a relationship exists between our features and targets, so the equation $f(x) ≈ y$ can be satisfied.


However, not all datasets are so easy! Havelte is a solar production plant, meaning that the power returned to the grid maps very closely to the total production in this site. Let's see if this holds true across different sites, by taking a look at the Return vs BackProduction for all of our available datasets:

In [ ]:
"""Iterate over all the dataset CSV files, plotting Return (in red) vs BackProduction (in green).
Takes a minute!"""
filepaths = glob.glob(data_dir+"*.csv")
dataset_list = []

for i,f in enumerate(filepaths):
  #Read current CSV into a Pandas DataFrame
  dataset = pd.read_csv(f, sep=';', decimal=',')

  #Add dataset (with path) to superset, so we only have to load them once.
  dataset_list.append([f, dataset])

  #variables for graphs
  name = f[len(data_dir):-4]
  X = list(range(len(dataset)))
  #Add plot of Return vs BackDelivery of the current dataset
  fig = go.Figure()
  fig.add_trace(go.Scatter(
                  x=X,
                  y=dataset['BackProduction'],
                  name=name + "Back Production",
                  line_color='mediumseagreen',
                  opacity=0.8))

  fig.add_trace(go.Scatter(
                  x=X,
                  y=dataset['Return'],
                  name="Return",
                  line_color='red',
                  opacity=0.8))
  

  #Make it fancy!
  fig.update_layout(go.Layout(
      title=dict(x=0.45),
      yaxis=dict(title='Energy (KWh)'),
      xaxis=dict(title='Timestep'),
      title_text=f"{name} - Back Production vs Return",
      hovermode="x",
      template="plotly_dark"))
  fig.show()



Uh-oh! Looks like Havelte isn't exactly a typical site; the Back Production of most sites is definitely correlated with their Return, but not as cleanly as in a pure solar production plant. Not to worry - the relationship still exists, we're just gonna have to get smarter with our feature data, as we'll see in the 'Feature Engineering' section below.



## Defining Scope & Limitations
We can also see that some of these sites lack any obvious $x \rightarrow y$ relationship. 
>In particular, Haarlem, Stiens, and both the Amsterdam sites are quite obviously unusable.

The real world cause of this is that the site is usually consuming 100% of the solar panel production in-house, rather than sending any back to the grid for us to measure. In these cases, Back Production is simply not predictable, and we will discontinue working with such sites.


It is important to be able to distinguish the trash from the treasure mathematically, rather than visual intuition. Furthermore, we must do so based on purely the features $x$, as we *will not* have the target $y$ when attempting to predict unseen sites.


The following code snippet works on the assumption that "trash" sites are outliers with respect to their inter-feature covariance.

>**Important Note:** Only *features* can be considered when determining whether a site will be predictable. The target column *should not* be used.

In [ ]:
"""Taking out the trash:
If the covariance coefficient between Delivery and Return exceeds a certain predefined threshold,
exlclude the dataset from further use."""
#Iterate over datasets
for path, dataset in dataset_list:
  #MinMaxScaler class - Transforms each column's values to lie within a given range.
  #This puts each independent dataset on 'equal footing' for calculating correlation.
  #Further detail on scaling is available in the Machine Learning section below, or at:
  #https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
  minmax_scaler = MinMaxScaler(feature_range=(0,1))

  features = ["Delivery", "Return"]
  scaled_dataset = minmax_scaler.fit_transform(dataset[features])

  #Divide-by-zero errors cause NaNs in Numpy, so trim zero value points.
  el_et = np.array([a for a in scaled_dataset if a[0] > 0 and a[1] > 0])
  
  #Find the 'magic number'
  covar = np.cov(el_et[:,0], el_et[:,1])[1,1]
  #Eliminate above threshold
  if covar > 0.0441:
    print("Site: {0};  Covariance = {1}".format(path[len(root_dir):-4], covar))

#Feature Engineering 🌤

---

> *The process of selecting, enriching and optimising features based on their usefulness to a predictive model.*

Through the preliminary investigation conducted in the previous section, we have determined that Delivery and Return are useful features for predicting the Back Production of sites within our scope. However, different sites have a wide range of energy profiles, making it difficult to produce a predictive model which can generalise to all cases using only these two features.

Luckily, we have domain knowledge! We know that in reality, **solar panel production is dependent upon the weather**. Primarily solar intensity, but a multitude of other factors could potentially be influential. In this section, we enrich site data with weather data from KNMI, and emprically veryify the usefulness of these new features.


 *Recommended Reading:* [*An Introduction to Variable and Feature Selection*](http://jmlr.csail.mit.edu/papers/volume3/guyon03a/guyon03a.pdf)


##Feature Enrichment - KNMI Weather Data

In this section, a new dataset is introduced. Data from the site named '171550' (as seen above) has been merged with climatology data, which has been collected from the geographically closest KNMI weather station. The code for parsing these two data sources has been omitted (it's not at all pretty), but sufficed to say;

> ***Each time step in the dataset now has 20 additional weather features!***

*For a full explanation of each of these features, see the official [KNMI documentation](https://projects.knmi.nl/klimatologie/uurgegevens/selectie.cgi).*

In [ ]:
#Set file directory for data
data_dir = root_dir + "Feature_Engineering/"
path = data_dir + "0_KNMI_merged.npy"

column_labels = ["HH","DD","FH","FF","FX","T","TD","SQ","Q","DR","RH","P","VV","N","U","IX","M","R","S","O","Y","EL","ET","BPM"]

dataset = np.load(path, allow_pickle=True)
dataframe = pd.DataFrame(dataset, columns=column_labels)

#Split into features (X) and targets (Y)
X = dataset[:,:-1]
Y = dataset[:,-1]

##Daily Aggregation
The business use case for this project is to predict solar production on a daily basis. KNMI also only updates their public data once per day, which puts a hard limit on response time. ***Therefore, the datasets have been aggregated from hourly into daily timesteps.***


> *Sadly, this means we have to eliminate the 'HH' feature! Although potentially useful, there is no logical way to encode the hour of the day into a daily timestep.*


On the bright side, aggregating in this way substantially reduces the noise and variability inherent to the data. This will allow our ML algorithm to converge faster and (hopefully) predict more accurately.

In [ ]:
#Remove HH column
X = dataset[:,1:-1]
Y = dataset[:,-1]

dataframe = dataframe.drop(columns=["HH"], axis=1)

#Adjust number of features
n_features = X.shape[1]

#Remove HH and BPM from column labels (for plotting)
feature_labels = column_labels[1:-1]

##Correlations & Heatmaps
Just like with our original dataset, let's investigate feature - target correlations, and plot some visualisations to gain better understanding of the new features we're working with.

In [ ]:
correlations = dataframe.corr()["BPM"]
"""Show correlation coefficients of features to the target BPM.
Remember that positive and negative correlations are equally useful for predictions!
Therefore, sorting by absolute value is more informative."""
abs_sorted_correlations = correlations[correlations.abs().sort_values().keys()][::-1]
print(abs_sorted_correlations)

#Pearson Correlation Heatmaps
ax = sns.heatmap(correlations.sort_values().to_frame()[::-1] , xticklabels=True, yticklabels=True, cmap=plt.cm.plasma)
plt.show()
ax = sns.heatmap(dataframe.corr(), xticklabels=True, yticklabels=True, cmap=plt.cm.plasma)
plt.show()

##Extra Trees Feature Importance
A fantastic method of measuring how good your features will be for prediction, is *by using them for prediction*. It is then possible to evaluate which features contributed most to the correct predictions.


>*This is an example of **embedded** feature selection - using a machine learning algorithm as part of the feature selection process.*

*Further Reading:*
*   [*ExtraTrees Feature Selection*](https://www.geeksforgeeks.org/ml-extra-tree-classifier-for-feature-selection/)
*   [*LASSO Feature Selection*](https://beta.vu.nl/nl/Images/werkstuk-fonti_tcm235-836234.pdf)

In [ ]:
#Take a small subset of the data to keep the tree complexity low
sample = np.array(random.sample(list(dataset), 10))

#Train / Fit model
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X = sample[:, :-1], y = sample[:, -1:])

#Plot
d = tree.plot_tree(clf, feature_names=column_labels[1:])

In [ ]:
# Building the model
#See: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html
extra_tree_forest = ExtraTreesRegressor(n_estimators=100, n_jobs=-1, 
                                        max_features=3) #Play with me! Maximum number of features to look simultaneously at per decision node.
# Training the model 
extra_tree_forest.fit(X, Y) 

#Extract Feature Importances
importances = extra_tree_forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in extra_tree_forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1][:]

labels = list()
for i in indices:
  labels.append(feature_labels[i])

#Plot importances
fig = px.bar(dataframe, x=labels, y=importances[indices],
             title="Feature Importance",
             color=importances[indices],
             color_continuous_scale=px.colors.sequential.Agsunset_r,
             )
#Make it fancy!
fig.update_layout(go.Layout(
    title=dict(x=0.55),
    xaxis=dict(title='Feature'),
    yaxis=dict(title='% Importance'),
    template="plotly_dark"))
fig.show()


##Feature Reduction
Using the intuition gained from our data interrogation techniques, we have a clear ranking for the importance of each feature. Some of the features are providing almost zero predictive weighting, and so can be discarded with no drawbacks.


Real world data concerns mean we should actively aim to eliminate useless features, as data itself has a cost in terms of runtime, storage space, and processing power. For this reason, the following features have been deemed above the threshold of importance, and will henceforth be the only ones included in code.

*  ***Q :***  *Global solar radiation (J / cm2)*
*  ***SQ :***  *Duration of sunshine (0.1 hours)*
*  ***U :***  *Relative humidity (percentage)*
*  ***T :***  *Temperature (0.1 degrees Celsius)*
*  ***EL :***  *Elektriciteit Levering (KWh), a.k.a Delivery*
*  ***ET :***  *Elektriciteit Teruglevering (KWh), a.k.a. Return*


#Machine Learning 🤖🧠

---

>*Using algorithms to perform specific tasks without using explicit instructions.*

The fun bit! Now that we all of our features decided and our data *almost* completely prepared, it's time to start working with machine learning models. All the ML models discussed here implement two important functions:

*  ***Fit :***  *Execute the algorithm on a specific dataset, so that it may attempt to learn the relationship $f(x) = y$ present in the data.*
*  ***Predict :***  *Use the relationship learned during fitting to produce a predicted value of $y$ for any given $x$.*


In [ ]:
"""Set data directory for ML section."""
data_dir = root_dir + "Training_Data/"

filepaths = glob.glob(data_dir + "*.npy")

"""Declare feature shape. In our case;
#[Q, SQ, U, T, EL, ET] => [BPM]"""
n_features = 6

##Train : Test : Validation Splitting
In order for our machine learning model to actually *learn*, we need to provide many correct examples of what we want it to do. The ML algorithm will iterate over these example datapoints (in [batches](https://medium.com/analytics-vidhya/when-and-why-are-batches-used-in-machine-learning-acda4eb00763)), make a prediction, and then compare the prediction to the actual target. By evaluating the error between these two vectors, the model can update itself to try and correct for these errors, and thus learns to make more accurate predictions.


However, when it comes to evaluation, *we cannot evaluate with data points used to train the model!* As the algorithm has already seen the actual target for these points, it has learnt the correct answer. This would give the false impression that our model performs extremely well, but that performance would fall apart when presented with new unseen data. In other words, that's cheating!


To avoid this [data leakage](https://machinelearningmastery.com/data-leakage-machine-learning/) problem, we split the data into three distinct groups:

* ***Training Set:*** *Used to train the ML model. (80% of dataset)*

* ***Testing Set:*** *Used to evaluate the performance of the ML model. Can be used during training to help optimise learning. (20% of dataset)*

* ***Validation Set:*** *Used for the final model evaluation. Must remain unseen by the model until all weights are fixed. (Complete data from one site is left out of training entirely, aka out-of-sample data)*

In [ ]:
"""This practise is simple to implement in code using list slicing, as shown here with an example Numpy matrix. 
This snippet is used as part of the 'split_scale_datasets' function described below."""

example_dataset = np.ones(shape=(1000, 7))
#Randomly shuffle
np.random.shuffle(example_dataset)

#Find indexes by dividing length
total_length = len(example_dataset)
train_index = math.floor(total_length * 0.8)

#Split into sets
train_set = example_dataset[:train_index]
test_set = example_dataset[train_index:]

print(f"Train set shape: {train_set.shape} \nTest set shape: {test_set.shape}\n")

##Scaling & Normalization

For most ML algorithms, this step is *critical*, for two main reasons. Firstly, to give features equal weighting in the training process. Secondly, it mitigates the [exploding / vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem).

This is easily solved with MinMax Scaling and Standard Scaling:
> ***MinMax***:  $X_{norm} = \frac{X - X_{min}}{X_{max}-X_{min}}$

> ***Standardize***:  $X_{z} = X - \overline{X} \over \sigma_{X}$


Scaling is *even more* important for our specific use case. Each site has a different energy profile, and so will ultimately have a variable scale coefficient. This will cause the model to learn converge towards the average scale of all datasets when taken together, *which causes predictions on individual sites to suffer* ***drastically!*** Formulaicly, this means for our problem:

>  $f(x) \propto y$

or,
> $f(x) = BPM * c$, where c is different for each real-world site.






---


**REDACTED**


---
The following code has been altered to obscure potentially proprietary information. The exact method of scaling has been removed from this public repository.


In [ ]:
"""Class for implementing the aforementioned 'magic scaling' equation.
Inherits from Sklearn's Scaler base class."""
class MagicScaler(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.magic_scale = 1

  #Determine magic scale
  def fit(self, X):
    """
    <REDACTED>
    Magic scale equation is proprietary information.
    It has been redacted from the public release.
    </REDACTED>
    """
    magic = 0.5
    self.magic_scale = 1 / magic
    return magic

  def transform(self, data):
    return data * self.magic_scale

  def fit_transform(self, X, data):
    """fits magic scale to X,
    then scales and returns data"""
    self.fit(X)
    return self.transform(data)

  def inverse_transform(self, data):
    return data / self.magic_scale

"""Function to load data from all provided filepaths,
concatenate them into a single dataset,
and perform appropriate scaling operations.

Returned data is ready to go straight into an ML model!"""
def split_scale_datasets(filepaths, train_split=0.8):
  print("Reading datasets from the following files:", filepaths)
  #Initialize arrays and explicitly-shaped empty tensors for concatenation
  X = np.empty(shape=(1, n_features))
  Y = np.empty(shape=(1))
  Xs = np.array([])
  Ys = np.array([])

  #Scale all features by their column's mean and std, then between range [0-1]
  std_scaler = StandardScaler(with_mean=True, with_std=True)
  minmax_scaler = MinMaxScaler(feature_range=(0, 1))
  
  #instantiate magic scaler implementation (discussed above)
  Y_magic_scaler = MagicScaler()

  #Agglomerate all datasets into single ND-array (Sorry for the mess! <3)
  for F in filepaths:
    f = np.load(F)
    if not (len(Xs) == 0):
      Xs = np.concatenate( (Xs,  minmax_scaler.fit_transform(std_scaler.fit_transform(f[:,:-1]))), axis=0) #Split X from dataset, scale, and append to master set
      Ys = np.concatenate( (Ys,  Y_magic_scaler.fit_transform(f[:,:-1], f[:,-1].reshape(-1,1))), axis=0) #Split Y from dataset, scale, and append to master list
    #Please ignore! - gross extra condition needed for the first iteration only.
    else:
      Xs = np.array(  minmax_scaler.fit_transform(std_scaler.fit_transform(f[:,:-1])))
      Ys = np.array( Y_magic_scaler.fit_transform(f[:,:-1], f[:,-1].reshape(-1,1)))
  print(f"Features shape: {Xs.shape};  Target shape:{Ys.shape}")

  #Find indexes by dividing length
  total_length = len(Xs)
  train_index = math.floor(total_length * train_split)

  #Split into train : test : val
  train_X = Xs[:train_index]
  train_Y = Ys[:train_index]

  test_X = Xs[train_index:]
  test_Y = Ys[train_index:]

  print(f"Train size: {train_Y.shape[0]};  Test size: {test_Y.shape[0]}")

  return train_X, train_Y, test_X, test_Y
  
#Call function to get prepared datasets
train_X, train_Y, test_X, test_Y = split_scale_datasets(filepaths, train_split=0.91)


##Artificial Neural Networks
Finally, we arrive at the actual solution used to achieve the mission statement; Artificial Neural Networks. This is a whole beast of a topic in its own right, so this notebook will only provide a rudimentary overview for brevity. There are however some key terms which cannot be done without:

*   ***Neuron:*** *Single unit of computation. Takes multiple inputs, does math to them, and produces a single output via an [activation function](https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html).*

*   ***Layer:*** *A collection of interconnected neurons. Takes a [tensor](https://www.tensorflow.org/guide/tensor) as an input and returns a new tensor as an output, potentially flowing into. (hence the name, TensorFlow!)*

*  ***Hyperparameter:*** *Variable factors when constructing and training the model, such as number of neurons in a layer. Think of them like 'settings'.*

*   ***Loss:*** *a measure of error between the predictions a model produces and the actual target values. In our case, [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) is the chosen metric. (lower = better)*





To learn more, here's a more in-depth [introductory article](https://towardsdatascience.com/machine-learning-for-beginners-an-introduction-to-neural-networks-d49f22d238f9) (but there are tons of resources out there!)

###Building a Brain
Well, defining the architecture of a model and constructing it in code, anyway. At an abstract level, you can interpreit the arrangement of layers and neurons as the way the model will "think". More complex models are needed to tackle more complex problems, but incurr drawbacks such as higher data cost and [overfitting](https://en.wikipedia.org/wiki/Overfitting).


Here we define about as simple a network as possible for tackling this task.

* ***Input layer(n=6):***  Same number of neurons as our number of features.
* **[*Dense*](https://keras.io/layers/core/) *hidden layer(n=256)*** :  A fully connected layer. Number of neurons is arbitrarily chosen for now. This is discussed in detail later in the Hyperparameter Optimisation section.
* Output layer(n=1):  Our target is a scalar value, so only one output neuron.

In [ ]:
#Using Keras, see: https://keras.io
#Returns a model with a compiled architecture and empty weights.
def build_nn_model(n_features):
  model = Sequential()
  #Input layer:
  #n_feature neurons -> Hidden Layer of 256 neurons
  model.add(Dense(256, input_shape=((n_features,)), activation='relu'))
  #Output layer:
  #256 Hidden -> 1 Output neuron (the shape of our target, i.e. a scalar value)
  model.add(Dense(1))

  model.compile(optimizer=Adam(lr=1e-3, decay=1e-4),
                loss=['mean_squared_error'])
  return model

#Construct model
model = build_nn_model(n_features)

#Print a summary of the model architecture
model.summary()

#Draw a pretty network graph!
graph_fname = 'network_diagram.gv'
ann_viz(model, title="Simple Model Brain", filename=graph_fname)
with open(graph_fname) as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

###Training a Model
Now that we have constructed a basic model, it's time to train it on our pre-processed datasets.


In [ ]:
"""Callbacks are functions that run on every training epoch. See: https://keras.io/callbacks/
The Checkpoint callback calculates the model's loss for predictions on the testing set.
The best model's weights are saved to be loaded after training is complete, 
as the final training epoch is not necissarily the most optimal model."""
simple_brain_path = "/simple_solar_brain.h5"
checkpoint_callback = ModelCheckpoint(filepath=simple_brain_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
n_epochs = 100

#Start training!
history = model.fit(x=train_X,
          y=train_Y,
          batch_size=64,
          validation_data=(test_X, test_Y),
          verbose=1,
          callbacks=[checkpoint_callback],
          epochs=n_epochs)
#Reload best model
model.load_weights(simple_brain_path)

In [ ]:
"""Visualise training history:"""
#Plot Losses
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(
                x=list(range(n_epochs)),
                y=history.history['loss'],
                name='Train Loss',
                line_color='red',
                opacity=0.8))

fig_acc.add_trace(go.Scatter(
                x=list(range(n_epochs)),
                y=history.history['val_loss'],
                name='Test Loss',
                line_color='green',
                opacity=0.8))
#Make it fancy!
fig_acc.update_layout(go.Layout(
    title=dict(x=0.45),
    yaxis=dict(title='Loss'),
    xaxis=dict(title='Epoch'),
    title_text="Training History - Loss",
    hovermode="x",
    template="plotly_dark"))
fig_acc.show()

###Making Predictions
> *Hooray, we have a trained ML model! Now to put it to use.*


However, you'll probably notice that the training graphs are... terrible. At the very least, the training loss should decay expontentially, telling us that the model was able to learn for at least a couple epochs.


Amazingly though, this simple model can still perform admirably when predicting on our validation set:

In [ ]:
#Load validation file
validation = np.load(root_dir + "Validation_Data/Emmeloord.npy")

#Split
val_X = validation[:,:-1]
val_Y = validation[:,-1]

#Scale all features by their column's mean and std, then between range [0-1]
Y_magic_scaler = MagicScaler()
Y_magic_scaler.fit(val_X)

std_scaler = StandardScaler(with_mean=True, with_std=True)
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

val_X = minmax_scaler.fit_transform(std_scaler.fit_transform(val_X))

raw_predictions = model.predict(val_X)

#Unscale to real values
predictions = Y_magic_scaler.inverse_transform(raw_predictions)

#Plot Predictions vs Actual
fig_1 = go.Figure()

fig_1.add_trace(go.Scatter(
                x=list(range(len(val_Y))),
                y=list(val_Y),
                name='Actual',
                line_color='green',
                opacity=0.8))

fig_1.add_trace(go.Scatter(
                x=list(range(len(val_Y))),
                y=list(predictions.ravel()),
                name='Predicted',
                line_color='red',
                opacity=0.8))

#Make it fancy!
fig_1.update_layout(go.Layout(
    title=dict(x=0.45),
    yaxis=dict(title='Energy (KWh)'),
    xaxis=dict(title='Timestep'),
    title_text="Predictions vs. Actual",
    hovermode="x",
    template="plotly_dark"))

fig_1.show()


###Convolutional Neural Networks (CNNs)
These are a specialised implementation of neural networks, which pay explicit attention to the spatial ordering of the input features. The typically use-case for CNNs is in the domain of image recognition, as the order of pixels within the image is fundamentally important. 

*Further Reading: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53*

In this case, we utilise this spatial information by transforming the input data into batches of a certain number of timesteps.



In [ ]:
"""Batch split train / test datasets.
Takes a 14 day timeslice; to predict today, 13 days of previous data are needed."""
batch_size = 14
#Split Train into Batches
X = []
Y = []
for i in range(batch_size, train_X.shape[0]):# - diff
    X.append(train_X[i-batch_size:i, :n_features])
    Y.append(train_Y[i])
train_X_batches, train_Y_batches = np.array(X), np.array(Y)

#Split Test into Batches
X = []
Y = []
for i in range(batch_size, test_X.shape[0]):
    X.append(test_X[i-batch_size:i, :n_features])
    Y.append(test_Y[i])
test_X_batches, test_Y_batches = np.array(X), np.array(Y)

print(test_X_batches.shape, test_Y_batches.shape)

In [ ]:
def build_solar_deep_cnn(features, batch_size=14):
  input_layer = Input([batch_size, features])

  #ConvNet Timeslice approach
  x = Conv1D(256, 3, padding='same', activation='relu')(input_layer)
  x = Conv1D(256, 3, padding='same', activation='relu')(x)
  x = Conv1D(256, 3, padding='same', activation='relu')(x)
  x = MaxPooling1D(pool_size=2, padding='same')(x)
  x = Dropout(0.2)(x)

  #Skip Connection
  skip = Flatten()(x)
  skip = Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(0.001,0.001))(skip)

  #ConvBlock2
  x = Conv1D(128, 3, padding='same', activation='relu')(x)
  x = Conv1D(128, 3, padding='same', activation='relu')(x)
  x = MaxPooling1D(pool_size=2, padding='same')(x)
  x = Dropout(0.2)(x)

  #ConvBlock3
  x = Conv1D(64, 3, padding='same', activation='relu')(x)
  x = MaxPooling1D(pool_size=2, padding='same')(x)
  x = Dropout(0.2)(x)

  #Conv -> Dense
  x = Flatten()(x)
  x = Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(0.001,0.001))(x)

  #Merge skip layer
  merged = Add()([x,skip])
  x = Dense(512, activation='relu', kernel_regularizer=regularizers.l1_l2(0.001,0.001))(merged)
  x = Dropout(0.4)(x)

  output_layer = Dense(1)(x)

  model = Model(inputs = input_layer, outputs = output_layer)

  model.compile(optimizer=Adam(lr=0.0001, decay=1e-4),
                loss=['mean_squared_error'])
  
  return model

solar_model = None
#Construct solar and print a network summary
solar_model = build_solar_deep_cnn(n_features)
solar_model.summary()

In [ ]:
"""Callbacks - Run every each training epoch"""
#TensorBoard logging -- Pretty visualisations for models & training progress
logdir = "/content/tensor_logs/"
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
#Reduces learning rate when the model ceases to improve over multiple epochs to really squeeze out an optimised solution
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.9,
                            patience=5, min_lr=1e-5)
solar_brain_dir = "/content/solar_brain.h5"
#Save best model from training
checkpoint_callback = ModelCheckpoint(solar_brain_dir, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#Only stop training once the model has completely converged
early_stopping = EarlyStopping(monitor='val_loss', patience=100, min_delta=1e-4)
n_epochs = 5000

"""Start training!"""
history = solar_model.fit(x=train_X_batches,
          y=train_Y_batches,  
          batch_size=256,
          validation_data=(test_X_batches, test_Y_batches),
          verbose=1,
          callbacks=[checkpoint_callback,  reduce_lr, early_stopping, tensorboard_callback],
          epochs=n_epochs)
#Reload best model
solar_model.load_weights(solar_brain_dir)

###Visualise Training


In [ ]:
"""Tensorboard: A lovely way to visualise loss and accuracy across training epochs, 
as well as cool stuff like an in-depth model architecture diagram.
(Colab Only)"""
%reload_ext tensorboard
%tensorboard --logdir="/content/tensor_logs"

In [ ]:
"""Visualise training history:"""
# #Plot Losses
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(
                x=list(range(n_epochs)),
                y=history.history['loss'],
                name='Train Loss',
                line_color='red',
                opacity=0.8))

fig_acc.add_trace(go.Scatter(
                x=list(range(n_epochs)),
                y=history.history['val_loss'],
                name='Test Loss',
                line_color='green',
                opacity=0.8))
#Make it fancy!
fig_acc.update_layout(go.Layout(
    title=dict(x=0.45),
    yaxis=dict(title='Loss'),
    xaxis=dict(title='Epoch'),
    title_text="Training History - Loss",
    hovermode="x",
    template="plotly_dark"))
fig_acc.show()

###Predictions


In [ ]:

solar_model.load_weights(solar_brain_dir)
#Load validation file
validation = np.load(root_dir + "Validation_Data/0.npy")

#Split
val_X = validation[:,:-1]
val_Y = validation[:,-1]

#Scale all features by their column's mean and std, then between range [0-1]
Y_magic_scaler = MagicScaler()
Y_magic_scaler.fit(val_X)

std_scaler = StandardScaler(with_mean=False, with_std=False)
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

val_X = minmax_scaler.fit_transform(std_scaler.fit_transform(val_X))

batch_size = 14
#Split Train into Batches
X = []
Y = []
for i in range(batch_size, val_X.shape[0]):# - diff
    X.append(val_X[i-batch_size:i, :n_features])
    Y.append(val_Y[i])
val_X_batches, val_Y_batches = np.array(X), np.array(Y)

raw_predictions = solar_model.predict(val_X_batches)

#Unscale to real values
predictions = Y_magic_scaler.inverse_transform(raw_predictions)

#Plot Predictions
fig_1 = go.Figure()

fig_1.add_trace(go.Scatter(
                x=list(range(len(val_Y_batches))),
                y=list(val_Y_batches.ravel()),
                name='Actual',
                line_color='green',
                opacity=0.8))

fig_1.add_trace(go.Scatter(
                x=list(range(len(val_Y_batches))),
                y=list(predictions.ravel()),
                name='Predicted',
                line_color='red',
                opacity=0.8))

#Make it fancy!
fig_1.update_layout(go.Layout(
    title=dict(x=0.45),
    yaxis=dict(title='Energy (KWh)'),
    xaxis=dict(title='Timestep'),
    title_text="Predictions vs. Actual",
    hovermode="x",
    template="plotly_dark"))

fig_1.show()
print(predictions.mean(),val_Y.mean())